In [ ]:
import os
import json
from collections import defaultdict

def preprocess_persona_dialogs_grouped_by_topic(json_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    topic_grouped_data = defaultdict(list)

    length = len(os.listdir(json_dir))
    count=0

    for filename in os.listdir(json_dir):
        count+=1
        if not filename.endswith(".json"):
            continue

        filepath = os.path.join(json_dir, filename)
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)


        personas = data.get("info", {}).get("personas", [])

        # 모든 페르소나가 성별과 연령대 모두 포함해야만 처리
        skip_file = False
        for p in personas:
            has_profile = any(
                prof.get("profile") is not None and prof.get("profile_major") is not None
                for prof in p.get("persona", [])
            )
            has_gender = any(
                prof.get("profile_major") == "성별" and prof.get("profile_minor") is not None
                for prof in p.get("persona", [])
            )
            has_age = any(
                prof.get("profile_major") == "연령대" and prof.get("profile_minor") is not None
                for prof in p.get("persona", [])
            )
            if not (has_profile and has_gender and has_age):
                skip_file = True
                break
        if skip_file:
            print(f"{filename} 스킵됨: 프로필 정보(성별, 연령대, ...) 중 하나라도 없음")
            continue
        

        raw_topic = data.get("info", {}).get("topic", "기타")
        topic = raw_topic.replace("/", ",")  # 파일명 안전하게

        # 페르소나 처리
        personas = data.get("info", {}).get("personas", [])
        persona_dict = {}
        for p in personas:
            pid = f"persona_{p['persona_id']}"
            profile_lines = []
            for prof in p["persona"]:
                major = prof.get("profile_major")
                if prof.get("profile_major") == "성별":
                    info = prof.get("profile_minor")
                elif prof.get("profile_major") == "연령대":
                    if  prof.get("profile_minor") =="10대 이하":
                        info = "10대"
                    if  prof.get("profile_minor") =="50대" or prof.get("profile_minor") =="60대 이상":
                        info = "50대이상"
                    else:
                        info = prof.get("profile_minor")
                else:
                    info = prof.get("profile", "")
                profile_lines.append(f"{major}:{info}")
            persona_dict[pid] = profile_lines

        # 대화 처리
        dialog_lines = []
        for utt in data.get("utterances", []):
            pid = f"persona_{utt['persona_id']}"
            dialog_lines.append(f"{pid}: {utt['text']}")

        # 하나의 대화 단위로 묶기
        dialogue_entry = {
            "personas": persona_dict,
            "dialog": dialog_lines
        }
        topic_grouped_data[topic].append(dialogue_entry)

        if(count%1000==0):
            print(f"{count}/{length}")
    print("페르소나를 같은 list 위치에 저장 완료")

    # 토픽별로 하나의 파일에 여러 대화 쓰기
    for topic, conversations in topic_grouped_data.items():
        output_path = os.path.join(output_dir, f"{topic}.json")
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(conversations, f, ensure_ascii=False, indent=2)
        print(f"{topic} json파일 생성 완료")
    print("페르소나 전처리 완료")

# 사용 예시
preprocess_persona_dialogs_grouped_by_topic(
    json_dir="raw_persona_dataset/training_dataset", 
    output_dir="preprocess_persona_dataset_new/training_dataset"
)
preprocess_persona_dialogs_grouped_by_topic(
    json_dir="raw_persona_dataset/validation_dataset", 
    output_dir="preprocess_persona_dataset_new/validation_dataset"
)
